In [60]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [12]:
import chardet

with open('C:/Users/user/Desktop/saltlux_project/ML_DL/preprocess/JungrangTotal_sentiment.csv', 'rb') as file:
    result = chardet.detect(file.read())

print(result['encoding'])

Windows-1252


In [99]:
# 데이터 로드
Jungrang_reviews_sentiment = pd.read_excel('../preprocess/Jungrang_sentiment.xlsx', index_col=0)
Jungrang_reviews_sentiment.head(2)

,name,hospital_id,nickname,content,date,revisit,sentiment
0,365mc모인이비인후과의원,19527111,Wiseburge David,사가정 모인 이비인후과 항상 만원이다,11.24.금,4번째 방문,0.0
1,365mc모인이비인후과의원,19527111,Dk9,좋아요,11.23.목,1번째 방문,1.0


In [100]:
Jungrang_reviews_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5975 entries, 0 to 9
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         5975 non-null   object 
 1   hospital_id  5975 non-null   int64  
 2   nickname     5975 non-null   object 
 3   content      5975 non-null   object 
 4   date         5975 non-null   object 
 5   revisit      5975 non-null   object 
 6   sentiment    5967 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 373.4+ KB


In [101]:
Jungrang_reviews_sentiment['sentiment'].isnull().sum()

8

In [102]:
Jungrang_reviews_sentiment = Jungrang_reviews_sentiment.reset_index(drop=True)

In [103]:
Jungrang_reviews_sentiment[Jungrang_reviews_sentiment['sentiment'].isnull()]

,name,hospital_id,nickname,content,date,revisit,sentiment
101,강북21세기병원,12856585,맛있어서개굴개굴,친절하십니다,11.16.목,4번째 방문,NaN
414,경희한의원,1376397047,목련8413,원장님과 직원분들 넘 친절해요~,22.8.22.월,1번째 방문,NaN
672,김영준내과의원,19514474,밍크커피,원장님 간호사님들 진료 잘봐주시고 친절하십니다,8.22.화,9번째 방문,NaN
674,김영준내과의원,19514474,수청9,ㅇㅇ,7.10.월,2번째 방문,NaN
874,늘찬병원,38287252,uhj****,팔다리가 너무 저려서 진료 받았는데.. 허리디스크 초기라네요. 여기 아니였으면 몰랐...,10.31.화,1번째 방문,NaN
950,다솔한의원,19523510,빠끔이10,치료후 가벼워집니다_x000D_\n,11.25.토,5번째 방문,NaN
1821,바른선택치과의원,1185355355,kru****,친절하시구 꼼꼼히 진료 잘 봐주십니다! 믿고 갈 수 있는 치과에요~ 항상 감사합니다!??,11.21.화,1번째 방문,NaN
5282,참빛소아청소년과의원,36476649,고기가좋아,마트안에 있어서 주차하기 편리해요.,10.26.목,34번째 방문,NaN


In [104]:
Jungrang_reviews_sentiment.loc[101, 'sentiment'] = 1
Jungrang_reviews_sentiment.loc[414, 'sentiment'] = 1
Jungrang_reviews_sentiment.loc[672, 'sentiment'] = 1
Jungrang_reviews_sentiment.loc[674, 'sentiment'] = 1
Jungrang_reviews_sentiment.loc[874, 'sentiment'] = 1
Jungrang_reviews_sentiment.loc[950, 'sentiment'] = 1
Jungrang_reviews_sentiment.loc[1821, 'sentiment'] = 1
Jungrang_reviews_sentiment.loc[5282, 'sentiment'] = 1

In [105]:
Jungrang_reviews_sentiment['sentiment'] = Jungrang_reviews_sentiment['sentiment'].astype(int)
Jungrang_reviews_sentiment

,name,hospital_id,nickname,content,date,revisit,sentiment
0,365mc모인이비인후과의원,19527111,Wiseburge David,사가정 모인 이비인후과 항상 만원이다,11.24.금,4번째 방문,0
1,365mc모인이비인후과의원,19527111,Dk9,좋아요,11.23.목,1번째 방문,1
2,365mc모인이비인후과의원,19527111,귀한집딸z,좋아요,11.20.월,2번째 방문,1
3,365mc모인이비인후과의원,19527111,꽃길걷는중임,항상 친절한 진료 감사합니다~,11.20.월,2번째 방문,1
4,365mc모인이비인후과의원,19527111,막둥,ㅇㅇ,11.18.토,4번째 방문,1
...,...,...,...,...,...,...,...
5970,효치과의원,19532002,fpvocalist,굳,9.20.수,13번째 방문,1
5971,효치과의원,19532002,소보루39,좋아요,9.20.수,11번째 방문,1
5972,효치과의원,19532002,1vvovv1,친절하십니다_x000D_\n과잉진료 없음,9.13.수,5번째 방문,1
5973,효치과의원,19532002,버럭아저씨,친절해요..,9.8.금,1번째 방문,1


In [ ]:
# 데이터 전처리
# 결측치 제거 -> null값 없음
# Jungrang_reviews_sentiment.dropna(inplace=True)

In [106]:
Jungrang_reviews_sentiment.duplicated().sum()

2

In [107]:
# 중복값 제거
Jungrang_reviews_sentiment = Jungrang_reviews_sentiment.drop_duplicates()
Jungrang_reviews_sentiment.shape

(5973, 7)

In [108]:
# 인덱스 재정렬
Jungrang_reviews_sentiment = Jungrang_reviews_sentiment.reset_index(drop=True)
Jungrang_reviews_sentiment.head(2)

,name,hospital_id,nickname,content,date,revisit,sentiment
0,365mc모인이비인후과의원,19527111,Wiseburge David,사가정 모인 이비인후과 항상 만원이다,11.24.금,4번째 방문,0
1,365mc모인이비인후과의원,19527111,Dk9,좋아요,11.23.목,1번째 방문,1


In [109]:
Jungrang_sentiment = Jungrang_reviews_sentiment[['name', 'content', 'sentiment']]
Jungrang_sentiment.head(2)

,name,content,sentiment
0,365mc모인이비인후과의원,사가정 모인 이비인후과 항상 만원이다,0
1,365mc모인이비인후과의원,좋아요,1


In [110]:
Jungrang_sentiment['content'] = Jungrang_sentiment['content'].str.replace('\n', ' ')
Jungrang_sentiment['content'] = Jungrang_sentiment['content'].str.replace('[~!]', ' ', regex=True) # regex=True : 정규표현식으로 해석(디폴트)
# Jungrang_sentiment['content'] = Jungrang_sentiment['content'].str.replace('[^가-힣]', ' ')
Jungrang_sentiment['content'] = Jungrang_sentiment['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백 제외하고 모두 제거
Jungrang_sentiment['content'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
print(Jungrang_sentiment.isnull().sum())

name         0
content      7
sentiment    0
dtype: int64


C:\Users\user\AppData\Local\Temp\ipykernel_2904\526288299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jungrang_sentiment['content'] = Jungrang_sentiment['content'].str.replace('\n', ' ')
C:\Users\user\AppData\Local\Temp\ipykernel_2904\526288299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jungrang_sentiment['content'] = Jungrang_sentiment['content'].str.replace('[~!]', ' ', regex=True) # regex=True : 정규표현식으로 해석(디폴트)
C:\Users\user\AppData\Local\Temp\ipykernel_2904\526288299.py:4: SettingWithCo

가-힣: 앞에서 언급한 것처럼 '가'부터 '힣' 사이의 모든 한글 음절을 매치시킵니다. 공백 문자도 포함되어 있습니다.
[^ㄱ-ㅎㅏ-ㅣ가-힣 ]는 자음, 모음, 한글 음절, 공백을 포함한 모든 문자를 제외한 나머지 문자를 매치시킵니다

[^가-힣]는 "안녕하세요123"에서 '1', '2', '3'을 제외한 모든 문자를 매치시킵니다.
[^ㄱ-ㅎㅏ-ㅣ가-힣 ]는 "안녕하세요123"에서 '1', '2', '3' 및 영어 알파벳과 같은 다른 문자도 제외한 모든 문자를 매치시킵니다.

In [111]:
Jungrang_sentiment = Jungrang_sentiment.dropna(how='any') # Null 값 제거
print('전처리 후 샘플의 개수 :',len(Jungrang_sentiment))

전처리 후 샘플의 개수 : 5966


In [112]:
from konlpy.tag import Okt
tokenizer = Okt()

In [113]:
stop_words = ['은', '는', '이', '가', '을', '를', '다', '의', '가', '이', '은', '한', '에', '하', '고', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
Jungrang_sentiment['tokenized'] = Jungrang_sentiment['content'].apply(tokenizer.morphs)
Jungrang_sentiment['tokenized'] = Jungrang_sentiment['tokenized'].apply(lambda x: [item for item in x if item not in stop_words])

In [114]:
Jungrang_sentiment.head(10)

,name,content,sentiment,tokenized
0,365mc모인이비인후과의원,사가정 모인 이비인후과 항상 만원이다,0,"[사가정, 모인, 이비인후과, 항상, 만원, 이다]"
1,365mc모인이비인후과의원,좋아요,1,[좋아요]
2,365mc모인이비인후과의원,좋아요,1,[좋아요]
3,365mc모인이비인후과의원,항상 친절한 진료 감사합니다,1,"[항상, 친절한, 진료, 감사합니다]"
4,365mc모인이비인후과의원,ㅇㅇ,1,[ㅇㅇ]
5,365mc모인이비인후과의원,ㅇㅇ,1,[ㅇㅇ]
6,365mc모인이비인후과의원,ㅇㅇ,1,[ㅇㅇ]
7,365mc모인이비인후과의원,ㅇㅇ,1,[ㅇㅇ]
8,365mc모인이비인후과의원,굿,1,[굿]
9,365mc모인이비인후과의원,굿,1,[굿]


In [115]:
# 데이터 분할 (학습 데이터와 테스트 데이터로 나눔)
train_data, test_data, train_labels, test_labels = train_test_split(
    Jungrang_sentiment['content'], Jungrang_sentiment['sentiment'], test_size=0.2, random_state=42)

In [116]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
train_data_tfidf = tfidf_vectorizer.fit_transform(train_data)
test_data_tfidf = tfidf_vectorizer.transform(test_data)

In [117]:
# 머신러닝 모델 학습
classifier = MultinomialNB() # 나이브 베이즈(Naive Bayes) 분류 모델, 주로 텍스트 분류 문제에 사용
classifier.fit(train_data_tfidf, train_labels)

MultinomialNB()

In [118]:
# 예측
predictions = classifier.predict(test_data_tfidf)

In [119]:
# 정확도 평가
accuracy = accuracy_score(test_labels, predictions)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.90


In [121]:
# 분류 보고서 출력
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       113
           1       0.90      1.00      0.95      1080
          11       0.00      0.00      0.00         1

    accuracy                           0.90      1194
   macro avg       0.30      0.33      0.32      1194
weighted avg       0.82      0.90      0.86      1194



c:\Users\user\Desktop\saltlux_project\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\Desktop\saltlux_project\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\Desktop\saltlux_project\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

## 예측

In [126]:
# 데이터 로드
Yeongdeungpo_reviews = pd.read_csv('../preprocess/naver_map_crawling_data_영등포구 병원_1reviews.csv')
Yeongdeungpo_reviews.head(3)

,name,category,review,date,revisit
0,건양의료재단 김안과병원,안과,보험수가,방문일\n11.25.토\n2023년 11월 25일 토요일,1번째 방문
1,건양의료재단 김안과병원,안과,...,방문일\n11.20.월\n2023년 11월 20일 월요일,6번째 방문
2,건양의료재단 김안과병원,안과,좋아요,방문일\n11.15.수\n2023년 11월 15일 수요일,1번째 방문


### 전처리 동일하게 적용

In [127]:
Yeongdeungpo_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16339 entries, 0 to 16338
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      16339 non-null  object
 1   category  16339 non-null  object
 2   review    16288 non-null  object
 3   date      16334 non-null  object
 4   revisit   16334 non-null  object
dtypes: object(5)
memory usage: 638.4+ KB


In [128]:
Yeongdeungpo_reviews = Yeongdeungpo_reviews[['name', 'review']]

In [130]:
# review null값 삭제
print(Yeongdeungpo_reviews['review'].isnull().sum())
Yeongdeungpo_reviews.dropna(inplace=True)
Yeongdeungpo_reviews.info()

51
<class 'pandas.core.frame.DataFrame'>
Index: 16288 entries, 0 to 16338
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    16288 non-null  object
 1   review  16288 non-null  object
dtypes: object(2)
memory usage: 381.8+ KB


In [131]:
Yeongdeungpo_reviews = Yeongdeungpo_reviews.reset_index(drop=True)
Yeongdeungpo_reviews.head(2)

,name,review
0,건양의료재단 김안과병원,보험수가
1,건양의료재단 김안과병원,...


In [132]:
Yeongdeungpo_reviews['review'] = Yeongdeungpo_reviews['review'].str.replace('\n', ' ')
Yeongdeungpo_reviews['review'] = Yeongdeungpo_reviews['review'].str.replace('[~!]', ' ', regex=True) # regex=True : 정규표현식으로 해석(디폴트)
# Jungrang_sentiment['content'] = Jungrang_sentiment['content'].str.replace('[^가-힣]', ' ')
Yeongdeungpo_reviews['review'] = Yeongdeungpo_reviews['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백 제외하고 모두 제거
Yeongdeungpo_reviews['review'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
print(Yeongdeungpo_reviews.isnull().sum())

name      0
review    0
dtype: int64


In [133]:
Yeongdeungpo_reviews = Yeongdeungpo_reviews.dropna(how='any') # Null 값 제거
print('전처리 후 샘플의 개수 :',len(Yeongdeungpo_reviews))

전처리 후 샘플의 개수 : 16288


In [135]:
stop_words = ['은', '는', '이', '가', '을', '를', '다', '의', '가', '이', '은', '한', '에', '하', '고', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
Yeongdeungpo_reviews['tokenized'] = Yeongdeungpo_reviews['review'].apply(tokenizer.morphs)
Yeongdeungpo_reviews['tokenized'] = Yeongdeungpo_reviews['tokenized'].apply(lambda x: [item for item in x if item not in stop_words])

In [136]:
Yeongdeungpo_reviews.head(10)

,name,review,tokenized
0,건양의료재단 김안과병원,보험수가,"[보험, 수가]"
1,건양의료재단 김안과병원,...,[...]
2,건양의료재단 김안과병원,좋아요,[좋아요]
3,건양의료재단 김안과병원,[1년째 다니는 중] 안과 대표 병원 중 한 곳이죠. 망막 병동이 아예 따로 있을 ...,"[[, 1년, 째, 다니는, 중, ], 안과, 대표, 병원, 중, 곳, 죠, ., ..."
4,건양의료재단 김안과병원,친절해요,[친절해요]
5,건양의료재단 김안과병원,굿,[굿]
6,건양의료재단 김안과병원,근래 다니는 중에 제일 대기자가 없던 진료였습니다. 그런데도 진료시간은 40분이나...,"[근래, 다니는, 중, 제일, 대, 기자, 없던, 진료, 였습니다, ., 그런데도,..."
7,건양의료재단 김안과병원,...,[...]
8,건양의료재단 김안과병원,좋아요,[좋아요]
9,건양의료재단 김안과병원,친절해요 ^^,"[친절해요, ^^]"


In [138]:
# 새로운 데이터에 대한 TF-IDF 벡터화
new_data_tfidf = tfidf_vectorizer.transform(Yeongdeungpo_reviews['review'])

# 학습된 모델을 사용하여 예측
new_predictions = classifier.predict(new_data_tfidf)

# 새로운 리뷰에 대한 감성 예측 결과를 DataFrame으로 변환
result_df = pd.DataFrame({'리뷰': Yeongdeungpo_reviews['review'], '예측 감성': new_predictions})

# CSV 파일로 저장
result_df.to_csv('../results/ML/prediction_results.csv', index=False, encoding='utf-8')

print("예측 결과가 'prediction_results.csv' 파일로 저장되었습니다.")

예측 결과가 'prediction_results.csv' 파일로 저장되었습니다.
